In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchattacks import PGD, DeepFool
import torch.optim as optim
from torch.utils.data import DataLoader

import Utils
import Models

print(torch.cuda.is_available())

True


In [2]:
def AdversarialAttack(Mynet, X, y, target, Iterations, learningRate, c):
    ListSavedX = []
    for Index in range(len(X)):
        x = X[Index].unsqueeze(0).clone()
        #x = torch.zeros_like(X[Index].unsqueeze(0))
        x = x.requires_grad_(True)
        MyOptimizer = Adam([x], lr=learningRate)
        
        SavedX = x
        PastLoss = 1000 
        for i in range(Iterations):
            goodness_per_label = []
            for label in range(10):
                z = Utils.overlay_y_on_x(x, label)
                goodness = []
                z = Mynet.Layer1(z)
                goodness += [z.pow(2).mean(1)]
                z = Mynet.Layer2(z)
                goodness += [z.pow(2).mean(1)]
                z = Mynet.Layer3(z)
                goodness += [z.pow(2).mean(1)]
                z = Mynet.Layer4(z)
                goodness += [z.pow(2).mean(1)]
                goodness_per_label += [sum(goodness).unsqueeze(1)]
            goodness_per_label = torch.cat(goodness_per_label, 1)

            _, Indexs = torch.topk(goodness_per_label[0], k=2)
            Pred = goodness_per_label[0]
            
            if target != -1:
                Current = Indexs[0]
                if Current == target:
                    Current = Indexs[1]       
                PredLoss = torch.max(Pred[y[Current]] - Pred[target], torch.tensor(-0.0001))
            else:
                Current = Indexs[0]
                if Current == y[Index]:
                    Current = Indexs[1]       
                PredLoss = torch.max(Pred[y[Index]] - Pred[Current], torch.tensor(-0.0001))

            Loss = PredLoss *c + Utils.L2Distance(x, X[Index])
            #print(i, Loss, PredLoss, Utils.L2Distance(x, X[Index]))
            
            if Loss < PastLoss and PredLoss < 0:
                SavedX = x.clone()
            
            MyOptimizer.zero_grad()
            Loss.backward()
            MyOptimizer.step()
            
            with torch.no_grad():
               x.clamp_(0,1)
            
        ListSavedX.append(SavedX.detach()[0])
    return torch.stack(ListSavedX)

def IterativeNewAttack(Mynet, X, y, target, Iterations, learningRate):
    c = 1000
    AdversarialImages = AdversarialAttack(Mynet, X, y, target, 0, learningRate, c)
    SavedVectors = torch.zeros_like(AdversarialImages)
    
    while c > 0.1:
        print("NET Adversarial with c = ", c)
        AdversarialImages = AdversarialAttack(Mynet, X, y, target, Iterations, learningRate, c)
        AdversarialImages = torch.clamp(AdversarialImages, 0, 1)
        AdversarialLabels = Mynet.predict(AdversarialImages)
        for i in range(len(AdversarialLabels)):
            if y[i].cpu() != AdversarialLabels[i].cpu() and Utils.L2Distance(SavedVectors[i], X[i]) > Utils.L2Distance(AdversarialImages[i], X[i]):
                SavedVectors[i] = AdversarialImages[i]     
        c/=2
    return SavedVectors

In [3]:
import torchvision.transforms as transforms

torch.manual_seed(1234)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader, test_loader = Utils.CIFAR10_loaders(test_batch_size=1000, train_batch_size=50000)
TestX, TestY = next(iter(test_loader))
TestX, TestY = TestX.cuda(), TestY.cuda()

Files already downloaded and verified
Files already downloaded and verified


In [4]:
MyFFModel = Models.Net_CIFAR10()
Epochs = 15

Utils.FFTrain(MyFFModel, train_loader, Epochs)
Utils.FFEvaluate(MyFFModel, test_loader, device)

Epoch  0
Epoch  1
Epoch  2
Epoch  3
Epoch  4
Epoch  5
Epoch  6
Epoch  7
Epoch  8
Epoch  9
Epoch  10
Epoch  11
Epoch  12
Epoch  13
Epoch  14
Accuracy NET: 39.26%


0.3926

In [5]:
MLPModel = Models.MLP_CIFAR10().to(device)
Epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MLPModel.parameters(), lr=0.0001)

Utils.Train(MLPModel, train_loader, device, criterion, optimizer, Epochs)
Utils.Evaluate(MLPModel, test_loader, device)

Epoch 1, Loss: 2.303054094314575
Epoch 2, Loss: 2.2957165241241455
Epoch 3, Loss: 2.288760185241699
Epoch 4, Loss: 2.28151535987854
Epoch 5, Loss: 2.273707389831543
Epoch 6, Loss: 2.2650794982910156
Epoch 7, Loss: 2.2556986808776855
Epoch 8, Loss: 2.2455172538757324
Epoch 9, Loss: 2.2344934940338135
Epoch 10, Loss: 2.2226192951202393
Epoch 11, Loss: 2.209847927093506
Epoch 12, Loss: 2.1963353157043457
Epoch 13, Loss: 2.1820642948150635
Epoch 14, Loss: 2.1671180725097656
Epoch 15, Loss: 2.151707649230957
Epoch 16, Loss: 2.1358954906463623
Epoch 17, Loss: 2.1197874546051025
Epoch 18, Loss: 2.1036479473114014
Epoch 19, Loss: 2.08774471282959
Epoch 20, Loss: 2.072281837463379
Epoch 21, Loss: 2.057251214981079
Epoch 22, Loss: 2.0428085327148438
Epoch 23, Loss: 2.029095411300659
Epoch 24, Loss: 2.0160624980926514
Epoch 25, Loss: 2.0036375522613525
Epoch 26, Loss: 1.9917374849319458
Epoch 27, Loss: 1.9803526401519775
Epoch 28, Loss: 1.9694806337356567
Epoch 29, Loss: 1.9590755701065063
Epoch 

0.4034

In [6]:
MyCNNModel = Models.MyCNN_CIFAR10().to(device)
criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(MyCNNModel.parameters(), lr=0.0001)
Epochs = 100

train_loader, _ = Utils.CIFAR10_loaders(test_batch_size=2, train_batch_size=256)
Utils.Train(MyCNNModel, train_loader, device, criterion2, optimizer2, Epochs)
Utils.Evaluate(MyCNNModel, test_loader, device)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.722154974937439
Epoch 2, Loss: 1.5078448057174683
Epoch 3, Loss: 1.3137098550796509
Epoch 4, Loss: 1.2363778352737427
Epoch 5, Loss: 1.4160664081573486
Epoch 6, Loss: 1.0976526737213135
Epoch 7, Loss: 1.404398798942566
Epoch 8, Loss: 1.1441686153411865
Epoch 9, Loss: 0.9955028295516968
Epoch 10, Loss: 1.0191867351531982
Epoch 11, Loss: 1.0300601720809937
Epoch 12, Loss: 1.0717909336090088
Epoch 13, Loss: 0.8615545034408569
Epoch 14, Loss: 0.6669005155563354
Epoch 15, Loss: 0.8057676553726196
Epoch 16, Loss: 0.8241368532180786
Epoch 17, Loss: 0.8569369316101074
Epoch 18, Loss: 0.8856935501098633
Epoch 19, Loss: 0.7659147381782532
Epoch 20, Loss: 0.48274222016334534
Epoch 21, Loss: 0.6481425166130066
Epoch 22, Loss: 0.6617892980575562
Epoch 23, Loss: 0.5729982256889343
Epoch 24, Loss: 0.4090944230556488
Epoch 25, Loss: 0.4278620183467865
Epoch 26, Loss: 0.4531365931034088
Epoch 27, Loss: 0.356432

In [ ]:
LoaderTest = DataLoader(list(zip(TestX, TestY)), batch_size=64, shuffle=False)
X_advFF = IterativeNewAttack(MyFFModel, TestX, TestY,  -1, 300, 0.1)  

attacks = {
    'PGD': PGD(MLPModel, eps=0.1, alpha=0.02),
    'DeepFool': DeepFool(MLPModel, steps=100, overshoot=0.02)
}

X_advDeep, _ = Utils.generate_adversarial_examples(MLPModel, LoaderTest, attacks['DeepFool'])
X_advDeep = torch.clamp(X_advDeep, 0, 1)

X_advPGD, _ = Utils.generate_adversarial_examples(MLPModel, LoaderTest, attacks['PGD'])
X_advPGD = torch.clamp(X_advPGD, 0, 1)

X_advCW = Utils.IterativeCW(MLPModel, TestX, TestY,  -1, 1000, 0.01)

Utils.DisplayResult(MLPModel, MyFFModel, MyCNNModel, device, TestX, TestY, X_advFF, X_advPGD, X_advCW, X_advDeep)
Utils.WriteResults("ResultsFFAdversrials_CIFAR10", MLPModel, MyFFModel, MyCNNModel, device, TestX, TestY, X_advFF, X_advPGD, X_advCW, X_advDeep)

NET Adversarial with c =  1000
NET Adversarial with c =  500.0
NET Adversarial with c =  250.0
NET Adversarial with c =  125.0
NET Adversarial with c =  62.5
NET Adversarial with c =  31.25
NET Adversarial with c =  15.625
NET Adversarial with c =  7.8125
NET Adversarial with c =  3.90625
NET Adversarial with c =  1.953125
NET Adversarial with c =  0.9765625
NET Adversarial with c =  0.48828125
NET Adversarial with c =  0.244140625
NET Adversarial with c =  0.1220703125
CW Adversarial with c =  1000
CW Adversarial with c =  500.0
CW Adversarial with c =  250.0
CW Adversarial with c =  125.0
CW Adversarial with c =  62.5
CW Adversarial with c =  31.25
CW Adversarial with c =  15.625
CW Adversarial with c =  7.8125
CW Adversarial with c =  3.90625
CW Adversarial with c =  1.953125
CW Adversarial with c =  0.9765625
CW Adversarial with c =  0.48828125
CW Adversarial with c =  0.244140625
CW Adversarial with c =  0.1220703125
CW Adversarial with c =  0.06103515625
CW Adversarial with c =  